In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
from imp import reload
import re
import pickle
import math, keras, datetime, pandas as pd, numpy as np, keras.backend as K
from pandas_summary import DataFrameSummary
import bld_etl; reload(bld_etl)
from keras.preprocessing import sequence
from bld_etl import read_raw, loadtypes, add_datepart, fillna, type_map, get_data
from sklearn.model_selection  import train_test_split

import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, Embedding, Concatenate
from keras import initializers, Model
from keras.layers.convolutional import Convolution1D
from keras.layers.pooling import MaxPooling1D
from keras import optimizers

In [6]:
def from_fasttest_to_vectors(file_name):
    with open(file_name, 'r') as f:
        data = f.readlines()
    samples, dim = data[0].split()
    samples, dim = int(samples), int(dim)
    wordidx = {}
    vecs = np.zeros((samples, dim), dtype='float32')
    for idx, line in enumerate(data[1:]):
        word, vec = line.split(' ', 1)
        vecs[idx] = np.array([ float(i) for i in vec.split()])
        wordidx[word] = idx
    return vecs, list(wordidx.keys()), wordidx

def dump_vectors(loc, vecs, words, wordidx):
    return (bld_etl.save_array(loc+'.dat', vecs),
        pickle.dump(words, open(loc+'_words.pkl','wb')),
        pickle.dump(wordidx, open(loc+'_idx.pkl','wb')) )


def load_vectors(loc):
    return (bld_etl.load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))
# dump_vectors('/root/.keras/models/fsttxtzh300', vecs, words, wordidx)
# vecs, words, wordidx = from_fasttest_to_vectors('/root/.keras/models/fsttxtzh300')

def create_emb(vocab_size, vecs, words):
    words_set = set(words_set)
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if re.match(r"^[A-Z]*$", word):
            word = word.lower()
        if word and (word in words_set):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

def par2idx(para, words):
    words_set = set(words)
    paras = []
    for word in para:
        if re.match(r"^[A-Z]*$", word):
            word = word.lower()
        if word and (word in words_set):
            paras.append(wordidx[word])
        else:
            paras.append(-1)
    return np.array(paras, dtype='int32')

In [7]:
vecs, words, wordidx = load_vectors('/root/.keras/models/fsttxtzh300')
idx2word = {v: k for k, v in wordidx.items()}

In [8]:
joined = pd.read_pickle('/data/result/cleaned_data.pkl')

In [9]:
data_df = joined.assign(ishigh=joined.TRANS_RBC > 3.5)[ ['OP_NAME', 'ishigh', 'TRANS_RBC']]
data_df.ishigh = data_df.ishigh.astype('int32')

In [10]:
corpus = [par2idx(line, words) for line in joined.OP_NAME]

In [11]:
vocab_size = 5000 # vocab_size = vecs.shape[0]

In [12]:
trn, test, y_train, y_test = train_test_split(corpus, joined.TRANS_RBC.values, test_size=0.2)

In [13]:
seq_len = 80

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

In [14]:
trn.shape

(4165, 80)

In [15]:
emb = vecs[:vocab_size]

In [16]:
model = Sequential([
    Embedding(vocab_size, 300, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='linear')])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, activation="relu", padding="same")`
  """


In [17]:
model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(), metrics=['accuracy'])

In [18]:
model.fit(trn, y_train, validation_data=(test, y_test), epochs=20, batch_size=64)

Train on 4165 samples, validate on 1042 samples
Epoch 1/20
4165/4165 [==============================] - 2s 391us/step - loss: 13.1700 - acc: 0.1885 - val_loss: 7.6949 - val_acc: 0.3100
Epoch 2/20
4165/4165 [==============================] - 0s 72us/step - loss: 8.0229 - acc: 0.1926 - val_loss: 7.8086 - val_acc: 0.3119
Epoch 3/20
4165/4165 [==============================] - 0s 66us/step - loss: 7.6613 - acc: 0.2067 - val_loss: 8.5891 - val_acc: 0.3580
Epoch 4/20
4165/4165 [==============================] - 0s 63us/step - loss: 7.5623 - acc: 0.2024 - val_loss: 8.3645 - val_acc: 0.3369
Epoch 5/20
4165/4165 [==============================] - 0s 63us/step - loss: 7.6226 - acc: 0.2053 - val_loss: 7.4216 - val_acc: 0.3225
Epoch 6/20
4165/4165 [==============================] - 0s 62us/step - loss: 7.2710 - acc: 0.2038 - val_loss: 6.2648 - val_acc: 0.1804
Epoch 7/20
4165/4165 [==============================] - 0s 63us/step - loss: 7.5517 - acc: 0.2108 - val_loss: 6.9188 - val_acc: 0.2620
Epoch

In [19]:
model.layers[0].trainable=True

In [21]:
model.optimizer.lr=1e-5

In [22]:
model.fit(trn, y_train, validation_data=(test, y_test), epochs=20, batch_size=64)

Train on 4165 samples, validate on 1042 samples
Epoch 1/20
2112/4165 [==============>...............] - ETA: 0s - loss: 5.4444 - acc: 0.2320

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


4165/4165 [==============================] - 0s 77us/step - loss: 5.5140 - acc: 0.2315 - val_loss: 6.2965 - val_acc: 0.2054
Epoch 2/20
4165/4165 [==============================] - 0s 65us/step - loss: 5.7532 - acc: 0.2238 - val_loss: 6.4225 - val_acc: 0.2111
Epoch 3/20
4165/4165 [==============================] - 0s 62us/step - loss: 5.5955 - acc: 0.2252 - val_loss: 6.9298 - val_acc: 0.2812
Epoch 4/20
4165/4165 [==============================] - 0s 59us/step - loss: 5.6416 - acc: 0.2370 - val_loss: 6.5080 - val_acc: 0.2198
Epoch 5/20
4165/4165 [==============================] - ETA: 0s - loss: 5.3535 - acc: 0.225 - 0s 61us/step - loss: 5.2847 - acc: 0.2276 - val_loss: 6.3902 - val_acc: 0.2111
Epoch 6/20
4165/4165 [==============================] - 0s 61us/step - loss: 5.4929 - acc: 0.2339 - val_loss: 6.1011 - val_acc: 0.1631
Epoch 7/20
4165/4165 [==============================] - 0s 57us/step - loss: 5.5326 - acc: 0.2312 - val_loss: 6.4340 - val_acc: 0.2121
Epoch 8/20
4165/4165 [======

In [24]:
predicted = model.predict(test)

In [3]:
diff0 = np.array(y_test).reshape(1, 1042)

NameError: name 'np' is not defined

In [54]:
diff1 = np.array(predicted).reshape(1, 1042)

In [2]:
diff = diff1 - diff0
diff = diff.flatten()

NameError: name 'diff1' is not defined

In [ ]:
diff[diff> 1] = 0
diff[diff< -1] = 0

In [76]:
np.count_nonzero(diff == 0)

601

In [ ]:
plt.hist(diff)

/usr/local/lib/python3.5/dist-packages/matplotlib/axes/_axes.py:6097: UserWarning: 2D hist input should be nsamples x nvariables;
 this looks transposed (shape is 1 x 1042)
  '(shape is %d x %d)' % inp.shape[::-1])


KeyboardInterrupt: 

Train on 4165 samples, validate on 1042 samples
Epoch 1/20
1984/4165 [=============>................] - ETA: 0s - loss: 0.6572 - acc: 0.5978

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


4165/4165 [==============================] - 0s 87us/step - loss: 0.6568 - acc: 0.6005 - val_loss: 0.6567 - val_acc: 0.5893
Epoch 2/20
4165/4165 [==============================] - 0s 80us/step - loss: 0.6531 - acc: 0.5935 - val_loss: 0.6568 - val_acc: 0.5883
Epoch 3/20
4165/4165 [==============================] - 0s 74us/step - loss: 0.6521 - acc: 0.6055 - val_loss: 0.6580 - val_acc: 0.5960
Epoch 4/20
4165/4165 [==============================] - 0s 82us/step - loss: 0.6498 - acc: 0.5990 - val_loss: 0.6540 - val_acc: 0.5950
Epoch 5/20
4165/4165 [==============================] - 0s 77us/step - loss: 0.6417 - acc: 0.6053 - val_loss: 0.6562 - val_acc: 0.6008
Epoch 6/20
4165/4165 [==============================] - 0s 91us/step - loss: 0.6480 - acc: 0.6086 - val_loss: 0.6534 - val_acc: 0.6046
Epoch 7/20
4165/4165 [==============================] - 0s 94us/step - loss: 0.6449 - acc: 0.6084 - val_loss: 0.6546 - val_acc: 0.5912
Epoch 8/20
4165/4165 [==============================] - 0s 94us/st